In [ ]:
#Import libs 
import pandas as pd
import numpy as np
import optuna
import statistics as st
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d
from scipy.stats import gmean
from scipy import stats
from scipy.stats import randint
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from scipy.spatial import Voronoi, voronoi_plot_2d
from IPython.display import display, HTML
import math
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as mlines
import plotly.graph_objects as go
import plotly.offline as go_offline
from plotly.subplots import make_subplots
import plotly.express as px
from tqdm import tqdm
import textwrap
import seaborn as sns
from statistics import mean
import geopandas as gpd
from shapely.geometry import Point, Polygon, mapping
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor 
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score as r2 
from sklearn.metrics import mean_absolute_error as mae 
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import make_scorer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
from yellowbrick.regressor import PredictionError
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from datetime import datetime
import random
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 15)
import warnings
warnings.filterwarnings('ignore')

## Func example

In [ ]:
def NetThicknessDistribution():

    def metadata_parquet_loading():
        path = 'C:\\jupyter\\SPP\\input\\'
        metadata_init = pd.read_csv(path + 'ACG_wells_metadata.csv', sep=',')
        metadata = metadata_init.copy()
        metadata = metadata.rename(columns={'X':'X_wellhead', 'Y':'Y_wellhead'})
        metadata.Status = metadata.Status.str.strip()
        metadata.Status = metadata.Status.str.lower()
        metadata.loc[metadata.Status == 'oil', 'Status' ] = 'production oil'
        metadata.loc[metadata.Status == 'oil producer', 'Status' ] = 'production oil'
        metadata.loc[metadata.Status == 'production', 'Status' ] = 'production oil'
        metadata.loc[metadata.Status == 'produiction oil', 'Status' ] = 'production oil'
        metadata.loc[metadata.Status == 'production_oil', 'Status' ] = 'production oil'
        metadata.loc[metadata.Status == 'abandoned production oil', 'Status' ] = 'abandoned oil'
        metadata.loc[metadata.Status == 'abandoned  oil', 'Status' ] = 'abandoned oil'
        metadata.loc[metadata.Status == 'abandoned oi', 'Status' ] = 'abandoned oil'
        metadata.loc[metadata.Status == 'injector  - water', 'Status' ] = 'injector - water'
        metadata.loc[metadata.Status == 'injector water', 'Status' ] = 'injector - water'
        metadata.loc[metadata.Status == 'injetor  - water', 'Status' ] = 'injector - water'
        metadata.loc[metadata.Status == 'abandoned injector - water per b', 'Status' ] = 'abandoned injector - water'
        metadata.loc[metadata.Status == 'plugged and abandoned', 'Status' ] = 'p&a'
        metadata.loc[metadata.X_wellhead==118.270, 'X_wellhead'] = 526258.84
        metadata.loc[metadata.Y_wellhead==526261.510, 'Y_wellhead'] = 4435802.01
        metadata.loc[metadata.well=='C39', 'X_wellhead'] = 526258.840
        metadata.loc[metadata.well=='C39', 'Y_wellhead'] = 4435802.010
        metadata.loc[metadata.field=='West Azeri', 'field'] = 'WEST AZERI'
        metadata.loc[metadata.field=='COP', 'field'] = 'WEST CHIRAG'
        metadata.loc[metadata.well=='AZERI2', 'field'] = 'WEST AZERI'
        metadata.loc[metadata.well=='AZERI3', 'field'] = 'WEST AZERI'
        metadata.loc[metadata.well=='B31', 'field'] = 'CENTRAL AZERI'
        metadata.loc[metadata.well=='J28_bpQIP', 'field'] = 'WEST CHIRAG'

        #Read data from parquet
        path = 'C:\\jupyter\\SPP\\input\\'
        df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v10.parquet.gzip')
        df_prq.rename(columns={'wellName':'well'}, inplace=True)
        df_prq = df_prq.set_index('well').join(metadata.set_index('well')).reset_index()
        # print('wells in df totally:', len(df_prq.well.unique()))
        # Filter data with bad_well_list 
        bad_well_list = ['E10Z','Predrill_J01Z', 'Predrill_J08', 'J28_bpQIP', 'A01W_2']
        df_prq = df_prq[~df_prq.well.isin(bad_well_list)]
        #Assign any Fluidcode_mod number by variable gross_pay=1 and gross_pay=0 if Fluidcode_mod as NaN
        df_prq.loc[df_prq.FLUIDS>0, 'FLUIDS_int'] = 1
        df_prq.loc[df_prq.FLUIDS<=0, 'FLUIDS_int'] = 0
        df_prq.FLUIDS_int = df_prq.FLUIDS_int.astype('int')
        # Unite of FU for each formation

        df_bal = df_prq[df_prq.FORMATION.str.contains('Balakhany')]
        df_bal.loc[df_bal.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
        df_bal.loc[df_bal.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
        df_bal = df_bal[df_bal.FORMATION_up.notna()]
        #Getting XY mean coords of Balakhany formation
        xy_coord_mean = df_bal[['well', 'FORMATION_up', 'X', 'Y']]
        xy_coord_mean = xy_coord_mean.groupby(['well', 'FORMATION_up']).agg({'X': 'mean', 'Y':'mean'}).reset_index()
        xy_coord_mean = xy_coord_mean.rename(columns={'X':'X_mean', 'Y':'Y_mean'})
        xy_coord_mean = xy_coord_mean[xy_coord_mean.FORMATION_up.str.contains('Balakhany') & (xy_coord_mean.X_mean>0) & (xy_coord_mean.Y_mean>0)]
        df_bal.rename(columns={'X':'X_traj', 'Y':'Y_traj'}, inplace=True)
        df_bal = df_bal.set_index(['well', 'FORMATION_up']).join(xy_coord_mean.set_index(['well', 'FORMATION_up'])).reset_index()
        return df_bal
    df_bal = metadata_parquet_loading()

    def well_clean_v2():
        #Counting of bad quality logs
        bal8_list = [   'Balakhany VIII sand', 'Balakhany VIII 20',   'Balakhany VIII 10',   'Balakhany VIII 25',
                        'Balakhany VIII 15',    'Balakhany VIII 5']
        well_tot8 = []
        well_zero8 = []
        well_clean8 = []
        for j in df_bal[(df_bal.FORMATION.isin(bal8_list) & (df_bal.PHIT>0))].well.unique():
            phit_zero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany VIII')]))
            phit_nonzero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany VIII') & (df_bal.PHIT > 0)]))
            well_tot8.append(j)
            if round((phit_nonzero/phit_zero),2)<=0.90:
                well_zero8.append(j)
            else:
                well_clean8.append(j)
                # well_display_ntd(df_bal, j, 'Balakhany VIII', 'NET', round((phit_nonzero/phit_zero),2), 1) #printing well plots with high quality logs

        bal10_list = ['Balakhany X', 'Balakhany X sand', 'Balakhany X 40', 'Balakhany X 20', 'Balakhany X 50']
        well_tot10 = []
        well_zero10 = []
        well_clean10 = []
        for j in df_bal[(df_bal.FORMATION.isin(bal10_list) & (df_bal.PHIT>0))].well.unique():
            phit_zero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany X')]))
            phit_nonzero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany X') & (df_bal.PHIT > 0)]))
            well_tot10.append(j)
            if round((phit_nonzero/phit_zero),2)<=0.90:
                # well_display_ntd(df_bal, j, 'Balakhany X', 'NET', round((phit_nonzero/phit_zero),2), 1)
                well_zero10.append(j)
            else:
                well_clean10.append(j)
        print('well_tot8', len(well_tot8))
        print('well_zero8', len(well_zero8))
        print('well_clean8', len(well_clean8))
        print('----------------------')
        print('well_tot10', len(well_tot10))
        print('well_zero10', len(well_zero10))
        print('well_clean10', len(well_clean10))

        # broken wells Bal8
        # A08, A19, H01Z, J05 
        # broken wells Bal10
        # C31, D25
        # high tst_interv Bal8
        # E30Z
        # small tst_interv Bal8
        # G01Z, E05, E01, E01Y, E11Z, E07, H01Y, H01Z, A14
        # Add wells after review Bal8 
        # D04Z, 
        # Remove wells from clean_list by any reasons
        remove_tst8 = ['A08','A19','J05','E30Z','G01Z', 'E05', 'E01', 'E01Y', 'E11Z', 'E07', 'H01Y', 'H01Z', 'A14']
        well_clean8_v2 = [i for i in well_clean8 if i not in remove_tst8]
        remove_tst10 = ['C31','D25', 'E21A']
        well_clean10_v2 = [i for i in well_clean10 if i not in remove_tst10]
        print('----------------------')
        print('well_clean8_v2: ', len(well_clean8_v2))
        print('well_clean10_v2: ', len(well_clean10_v2))
        return well_clean8_v2, well_clean10_v2
    well_clean8_v2, well_clean10_v2 = well_clean_v2()

    # Limitation dataframe to cleaned wells for Bal8 & Bal10
    df_net_bal8 = df_bal[['well', 'MD', 'TST', 'NET', 'FORMATION_up', 'LPERM', 'PHIT', 'VSH']]
    df_net_bal8 = df_net_bal8[df_net_bal8.well.isin(well_clean8_v2) & (df_net_bal8.FORMATION_up=='Balakhany VIII')]
    df_net_bal10 = df_bal[['well', 'MD', 'TST', 'NET', 'FORMATION_up', 'LPERM', 'PHIT', 'VSH']]
    df_net_bal10 = df_net_bal10[df_net_bal10.well.isin(well_clean10_v2) & (df_net_bal10.FORMATION_up=='Balakhany X')]
    # Calculation dataframe with h_tst from MD to TST for Bal8

    def ntd_calculation_brief(dataset,well,desired_fm, net_var='NET'):
        data = dataset[(dataset.well==well) & (dataset.FORMATION_up==desired_fm)]
        data.iloc[0, 3] = 0
        data.iloc[-1, 3] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = desired_fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_list8 = []
    print('Calculation dataframe with h_tst from MD to TST for Bal8')
    for well in tqdm(df_net_bal8.well.unique()):
        df = ntd_calculation_brief(df_net_bal8, well, 'Balakhany VIII', 'NET')
        df_list8.append(df)
    ntd_net_8 = pd.concat(df_list8)
    df_list10 = []
    print('Calculation dataframe with h_tst from MD to TST for Bal10')
    for well in tqdm(df_net_bal10.well.unique()):
        df = ntd_calculation_brief(df_net_bal10, well, 'Balakhany X', 'NET')
        df_list10.append(df)
    ntd_net_10 = pd.concat(df_list10)
    ntd_net_final = pd.concat([ntd_net_8, ntd_net_10])

    # Cleaning NET variable and making up NET_clp with clipped data, join NET_clp to main dataframe
    def ntd_htst_cleaning(dataset, cutoff):
        """
        dataset - any updated dataset like df_bal...
        cutoff - value in TST to remove layers with thickness below cutoff
        """
        df_list_ntd = []
        for well in tqdm(dataset.well.unique()):
            ntd_well = dataset[(dataset.well ==well)]
            ntd_well_cutoff = ntd_well[ntd_well.h_tst >= cutoff]
            well_short = df_bal[['well', 'FORMATION_up', 'MD', 'TST', 'GR_N', 'NET', 'FORMATION']]
            net_well = well_short[(well_short.well==well)]
            net_well['NET_clp'] = 0
            for j in range(len(ntd_well_cutoff.well)):
                ntd_top = ntd_well_cutoff.iloc[j, 2].round(3)
                ntd_bot = ntd_well_cutoff.iloc[j, 3].round(3)
                for i in range(len(net_well.TST)):
                    well_tst = net_well['TST'].iloc[i].round(3)
                    if well_tst >= ntd_top and well_tst <= ntd_bot:
                        net_well['NET_clp'].iloc[i] = 1
            df_list_ntd.append(net_well)
        net_clp = pd.concat(df_list_ntd)
        return net_clp
    print('Cleaning NET variable and making up NET_clp with clipped data')
    net_clp =  ntd_htst_cleaning(ntd_net_final, 1)
    df_bal_net = df_bal.set_index(['well','MD']).join(net_clp.drop(
        ['FORMATION_up','NET','TST', 'FORMATION', 'GR_N'], axis=1).set_index(['well','MD'])).reset_index()
    df_bal_net = df_bal_net[df_bal_net.NET_clp.notna()]

    # Cleaning NET_clp from 1-point zero
    print('Cleaning NET_clp from 1-point zero')
    for i in tqdm(range(len(df_bal_net.NET_clp))):
        if (df_bal_net.NET_clp.iloc[i] == 0 and  
            df_bal_net.NET_clp.iloc[i-1] == 1 and 
            df_bal_net.NET_clp.iloc[i+1] == 1):
            df_bal_net.NET_clp.iloc[i] = 1        

    df_zero_bal = df_bal_net[['well', 'MD', 'TST', 'NET_clp', 'FORMATION_up']]
    df_zero_bal8 = df_zero_bal[df_zero_bal.well.isin(well_clean8_v2) & (df_zero_bal.FORMATION_up=='Balakhany VIII')]
    df_zero_bal10 = df_zero_bal[df_zero_bal.well.isin(well_clean10_v2) & (df_zero_bal.FORMATION_up=='Balakhany X')]
    
    def ntd_calculation_zero(dataset,well,formation, net_var='NET'):
        data = dataset[(dataset.well==well) & (dataset.FORMATION_up==formation)]
        data.iloc[0, 3] = 1
        data.iloc[-1, 3] = 1
        tst_zero_top = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 0 and data.iloc[i-1][net_var] == 1)]
        tst_zero_bot = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1) 
                    if (data.iloc[i][net_var] == 0 and data.iloc[i+1][net_var] == 1)]
        tops_zero = zip(tst_zero_top, tst_zero_bot)
        df_zero_htst = pd.DataFrame(tops_zero, columns=['tst_zero_top', 'tst_zero_bot'])
        df_zero_htst['FORMATION_up'] = formation
        df_zero_htst['well'] = well
        df_zero_htst['h_tst_zero'] = df_zero_htst.tst_zero_bot - df_zero_htst.tst_zero_top
        df_zero_htst = df_zero_htst[['well','FORMATION_up','tst_zero_top','tst_zero_bot','h_tst_zero']]
        return df_zero_htst
    print('NET-zero layers removing Bal8')
    df_zero_list8 = []
    for well in tqdm(df_zero_bal8.well.unique()):
        df = ntd_calculation_zero(df_zero_bal8, well, 'Balakhany VIII', 'NET_clp')
        df_zero_list8.append(df)
    ntd_zero_8 = pd.concat(df_zero_list8)
    
    print('NET-zero layers removing Bal10')
    df_zero_list10 = []
    for well in tqdm(df_zero_bal10.well.unique()):
        df = ntd_calculation_zero(df_zero_bal10, well, 'Balakhany X', 'NET_clp')
        df_zero_list10.append(df)
    ntd_zero_10 = pd.concat(df_zero_list10)
    
    ntd_zero = pd.concat([ntd_zero_8, ntd_zero_10])

    # NET-zero layers removing
    print('Run ntd_htst_zero_cleaning')
    def ntd_htst_zero_cleaning(dataset_zero, dataset, cutoff, net_var1, net_var2):
        df_list_ntd_zero = []
        for well in tqdm(dataset_zero.well.unique()):
            ntd_well_zero = dataset_zero[(dataset_zero.well ==well)]
            ntd_well_zero_sel = ntd_well_zero[ntd_well_zero.h_tst_zero <= cutoff]
            well_zero_short = dataset[['well','FORMATION_up','MD','TST', net_var1, 'GR_N', 'NET', 'FORMATION']]
            well_zero_short[net_var2] = well_zero_short[net_var1]
            well_zero_sel = well_zero_short[(well_zero_short.well==well)]
            for j in range(len(ntd_well_zero_sel.well)):
                ntd_zero_top = ntd_well_zero_sel.iloc[j, 2].round(3)
                ntd_zero_bot = ntd_well_zero_sel.iloc[j, 3].round(3)
                for i in range(len(well_zero_sel.TST)):
                    well_zero_tst = well_zero_sel['TST'].iloc[i].round(3)
                    if well_zero_tst >= ntd_zero_top and well_zero_tst <= ntd_zero_bot:
                        well_zero_sel[net_var2].iloc[i] = 1
            df_list_ntd_zero.append(well_zero_sel)
        result = pd.concat(df_list_ntd_zero)
        return result
    net_clp2 = ntd_htst_zero_cleaning(ntd_zero, df_bal_net, 1, 'NET_clp', 'NET_clp2')

    #Joining NET_clp2 to main dataframe df_bal_net
    df_bal_net2 = df_bal_net.set_index(['well','MD']).join(net_clp2.drop(
        ['FORMATION_up','GR_N', 'NET','NET_clp', 'FORMATION','TST'], axis=1).set_index(['well','MD'])).reset_index()
    df_bal_net2 = df_bal_net2[df_bal_net2.NET_clp2.notna()]

    # KHtst calculation and join to the main dataframe df_bal_net2
    def calculation_khtst(dataset, net_var):
        df_smpl_lst = []
        print('TST sampling calculation')
        for well_smpl in tqdm(dataset.well.unique()[:]):
            tst_sampl = dataset[dataset.well==well_smpl]['TST'].diff()
            df_new = dataset[dataset.well==well_smpl].join(tst_sampl, rsuffix='_smpl')    
            df_smpl_lst.append(df_new)
        df_bal_tst_smpl = pd.concat(df_smpl_lst)
        df_kh_lst_fm = []
        print('KHtst calculation')
        for fm_kh in ['Balakhany VIII', 'Balakhany X']:
            df_kh_lst = []
            for well_kh in tqdm(dataset.well.unique()[:]):
                well_tst_perm = df_bal_tst_smpl[(df_bal_tst_smpl.well==well_kh) & 
                                                (df_bal_tst_smpl.FORMATION_up==fm_kh)].sort_values(by='MD', ascending=False)
                well_tst_perm.loc[well_tst_perm[net_var] == 0, 'LPERM'] = 0
                well_tst_perm.loc[well_tst_perm[net_var] == 0, 'PHIT'] = 0
                well_tst_perm.loc[well_tst_perm[net_var] == 0, 'VSH'] = 0
                well_tst_perm['khtst'] = well_tst_perm.LPERM*well_tst_perm.TST_smpl
                well_tst_perm['phithtst'] = well_tst_perm.PHIT*well_tst_perm.TST_smpl
                well_tst_perm['vshhtst'] = well_tst_perm.VSH*well_tst_perm.TST_smpl
                well_tst_perm['KHtst'] = well_tst_perm.khtst.cumsum()
                well_tst_perm['PHITHtst'] = well_tst_perm.phithtst.cumsum()
                well_tst_perm['VSHHtst'] = well_tst_perm.vshhtst.cumsum()
                well_tst_perm = well_tst_perm.sort_values(by='MD')
                df_kh_lst.append(well_tst_perm)
            df_khlst = pd.concat(df_kh_lst)
            df_kh_lst_fm.append(df_khlst)
        df_khlst_fm = pd.concat(df_kh_lst_fm)
        # df_khlst_fm = df_khlst_fm.dropna()
        return df_khlst_fm[['well', 'FORMATION_up', 'MD', 'TST', 'TST_smpl','KHtst','PHITHtst','VSHHtst']]
    df_kh = calculation_khtst(df_bal_net2, 'NET_clp2')
    df_bal_net2_kh_init = df_bal_net2.set_index(['well','MD']).join(df_kh.drop(['FORMATION_up', 'TST'], axis=1).set_index(['well','MD'])).reset_index()
    df_bal_net2_kh8 = df_bal_net2_kh_init[(df_bal_net2_kh_init.well.isin(well_clean8_v2)) & (df_bal_net2_kh_init.FORMATION_up == 'Balakhany VIII')]
    df_bal_net2_kh10 = df_bal_net2_kh_init[(df_bal_net2_kh_init.well.isin(well_clean10_v2)) & (df_bal_net2_kh_init.FORMATION_up == 'Balakhany X')]
    df_bal_net2_kh = pd.concat([df_bal_net2_kh8, df_bal_net2_kh10])

    # Displaying of ramdom well for example
    def well_display_net(dataset, well, formation, net1='NET_clp', net2_flag=0, net2='NET_clp_v2'):
        well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
        depth = well_sel['TST']
        grn = well_sel['GR_N']
        net = well_sel['NET']
        net_clp = well_sel[net1]
        if net2_flag == 0:
            fig, ax = plt.subplots(1,3, figsize=(4.5,8), sharey=True)
            ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), 5))
            ax[0].plot(grn, depth, color='green'), ax[0].invert_yaxis(), ax[0].set_xlim(0, 150), ax[0].grid(axis='y')
            well_bal_tops = well_sel.groupby('FORMATION')['TST'].apply(lambda x: x.iloc[0]).reset_index()
            for i in well_bal_tops[well_bal_tops.FORMATION.str.contains('Balakhany VIII')].FORMATION:
                ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0], xmin=0, xmax=150, color='black', lw=2, alpha=0.66)
                ax[0].text(10, well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0]+3, i, fontsize = 7, color ="black")
            ax[1].plot(net, depth, color='orange'), ax[1].set_xlim(0, 1), ax[1].grid(axis='y')
            ax[1].fill_betweenx(depth,net, color='orange', alpha=0.33)
            ax[2].plot(net_clp, depth, color='orange'), ax[2].set_xlim(0, 1), ax[2].grid(axis='y')
            ax[2].fill_betweenx(depth,net_clp, color='orange', alpha=0.33)
            fig.suptitle(well_sel.well.unique()[0], fontsize=14)
            fig.tight_layout()
        if net2_flag == 1:
            net_clp2 = well_sel[net2]
            fig, ax = plt.subplots(1,4, figsize=(6,8), sharey=True)
            ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), 5))
            ax[0].plot(grn, depth, color='green'), ax[0].invert_yaxis(), ax[0].set_xlim(0, 150), ax[0].grid(axis='y')
            well_bal_tops = well_sel.groupby('FORMATION')['TST'].apply(lambda x: x.iloc[0]).reset_index()
            for i in well_bal_tops[well_bal_tops.FORMATION.str.contains('Balakhany VIII')].FORMATION:
                ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0], xmin=0, xmax=150, color='black', lw=2, alpha=0.66)
                ax[0].text(10, well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0]+3, i, fontsize = 7, color ="black")
            ax[1].plot(net, depth, color='orange', lw=0.25), ax[1].set_xlim(0, 1), ax[1].grid(axis='y')
            ax[1].fill_betweenx(depth,net, color='orange', alpha=0.33)
            ax[2].plot(net_clp, depth, color='orange', lw=0.25), ax[2].set_xlim(0, 1), ax[2].grid(axis='y')
            ax[2].fill_betweenx(depth,net_clp, color='orange', alpha=0.33)
            ax[3].plot(net_clp2, depth, color='orange', lw=0.25), ax[3].set_xlim(0, 1), ax[3].grid(axis='y')
            ax[3].fill_betweenx(depth,net_clp2, color='orange', alpha=0.33)
            fig.suptitle(well_sel.well.unique()[0], fontsize=14)
            fig.tight_layout()
        return fig.show()
    well_display_net(df_bal_net2_kh, 'J28', 'Balakhany VIII', 'NET_clp', 1, 'NET_clp2')
    return df_bal_net2_kh

# df_bal_net2_kh = NetThicknessDistribution()

## Normal data upload

In [132]:
def data_upload(path, filename):
    df = pd.read_csv(path + filename)
    df = df.rename(columns={'wellName':'well', 'FORMATION_JOIN_BEST':'FORMATION', 'FORMATION_TOPS_AZPU':'TOPS_AZPU'})
    df['X_traj'] = df['SurfaceX'] + df['XOFFSET']
    df['Y_traj'] = df['SurfaceY'] + df['YOFFSET']
    df = df[df.FORMATION.notna()]
    df.loc[df.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
    df.loc[df.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
    df = df.astype({'well':'string', 'FORMATION':'string', 'FORMATION_up':'string', 'TOPS_AZPU':'string', 'CurrentType':'string', 'Field':'string'})
    xy_coord_mean = df.groupby(['well', 'FORMATION_up']).agg({'X_traj': 'mean', 'Y_traj':'mean'}).reset_index()
    xy_coord_mean = xy_coord_mean.rename(columns={'X_traj':'X_mean', 'Y_traj':'Y_mean'})
    df = df.set_index(['well', 'FORMATION_up']).join(xy_coord_mean.set_index(['well', 'FORMATION_up'])).reset_index()
    df.insert(3, 'FORMATION_up',df.pop('FORMATION_up'))
    df.insert(28, 'X_traj',df.pop('X_traj'))
    df.insert(29, 'Y_traj',df.pop('Y_traj'))
    df.insert(30, 'X_mean',df.pop('X_mean'))
    df.insert(31, 'Y_mean',df.pop('Y_mean'))
    return df
df = data_upload('C:\\jupyter\\SPP\\input\\', 'newTechlogData_allWells.csv')

In [ ]:
def interpolate_by_depth_fm_run(df, step):
    df_tst = df[df.TST.notna()].round({'MD':1})
    
    def interpolate_by_depth_fm(one_well, step):
        one_well = one_well.sort_values(by='TST')
        well_name = one_well["well"].iloc[0]
        data_range = np.floor((one_well["TST"].max() - one_well["TST"].min())/step)
        starting_tst = one_well["TST"].iloc[0]
        new_TST_values = [starting_tst + i*0.1 for i in range(1,int(data_range))]
        interp_MD = interp1d(one_well['TST'], one_well['MD'], kind='linear', fill_value="extrapolate")
        interp_DEVI = interp1d(one_well['TST'], one_well['DEVI'], kind='linear', fill_value="extrapolate")
        interp_HAZI = interp1d(one_well['TST'], one_well['HAZI'], kind='linear', fill_value="extrapolate")
        interp_NET = interp1d(one_well['TST'], one_well['NET'], kind='linear', fill_value="extrapolate")
        interp_NET_VSH = interp1d(one_well['TST'], one_well['NET_VSH'], kind='linear', fill_value="extrapolate")
        # interp_FLUIDS = interp1d(one_well['TST'], one_well['FLUIDS'], kind='linear', fill_value="extrapolate")
        interp_LPERM = interp1d(one_well['TST'], one_well['LPERM'], kind='linear', fill_value="extrapolate")
        interp_PHIT = interp1d(one_well['TST'], one_well['PHIT'], kind='linear', fill_value="extrapolate")
        interp_GR = interp1d(one_well['TST'], one_well['GR_N'], kind='linear', fill_value="extrapolate")
        interp_VSH = interp1d(one_well['TST'], one_well['VSH'], kind='linear', fill_value="extrapolate")
        interp_NPSS = interp1d(one_well['TST'], one_well['NPSS'], kind='linear', fill_value="extrapolate")
        interp_RHOB = interp1d(one_well['TST'], one_well['RHOB'], kind='linear', fill_value="extrapolate")
        interp_RDEEP = interp1d(one_well['TST'], one_well['RDEEP'], kind='linear', fill_value="extrapolate")
        interp_SON = interp1d(one_well['TST'], one_well['SON'], kind='linear', fill_value="extrapolate")
        interp_SONSH = interp1d(one_well['TST'], one_well['SONSH'], kind='linear', fill_value="extrapolate")
        interp_TVDSCS = interp1d(one_well['TST'], one_well['TVD_SCS'], kind='linear', fill_value="extrapolate")
        interp_Xtraj = interp1d(one_well['TST'], one_well['X_traj'], kind='linear', fill_value="extrapolate")
        interp_Ytraj = interp1d(one_well['TST'], one_well['Y_traj'], kind='linear', fill_value="extrapolate")
        interp_Xmean = interp1d(one_well['TST'], one_well['X_mean'], kind='linear', fill_value="extrapolate")
        interp_Ymean = interp1d(one_well['TST'], one_well['Y_mean'], kind='linear', fill_value="extrapolate")    
        # Create a new DataFrame with the interpolated values for new TVD_SCS
        new_data = {
            'well': [well_name for _ in range(len(new_TST_values))],
            # 'FORMATION_up': [formation_name for _ in range(len(new_TST_values))],
            'tst_index': [_ for _ in range(len(new_TST_values))],
            'MD': interp_MD(new_TST_values),
            'TST': new_TST_values,
            'DEVI': interp_DEVI(new_TST_values),
            'HAZI': interp_HAZI(new_TST_values),
            'NET': interp_NET(new_TST_values),
            'NET_VSH': interp_NET_VSH(new_TST_values),
            # 'FLUIDS': interp_FLUIDS(new_TST_values),
            'LPERM': interp_LPERM(new_TST_values),
            'PHIT': interp_PHIT(new_TST_values),
            'GR_N': interp_GR(new_TST_values),
            'VSH': interp_VSH(new_TST_values),
            'NPSS': interp_NPSS(new_TST_values),
            'RHOB': interp_RHOB(new_TST_values),
            'RDEEP': interp_RDEEP(new_TST_values),
            'SON': interp_SON(new_TST_values),
            'SONSH': interp_SONSH(new_TST_values),
            'TVD_SCS': interp_TVDSCS(new_TST_values),
            'X_traj': interp_Xtraj(new_TST_values),
            'Y_traj': interp_Ytraj(new_TST_values),
            'Xmean': interp_Xmean(new_TST_values),
            'Ymean':interp_Ymean(new_TST_values)
        }
        new_df = pd.DataFrame(new_data)
        return new_df
    df_lst = []
    print('Start interpolation')
    for wellnames in tqdm(df_tst.well.unique()):
        well_sel = df_tst[df_tst.well == wellnames]
        well_interp = interpolate_by_depth_fm(well_sel, step)
        df_lst.append(well_interp)
    df_interp = pd.concat(df_lst)
    df_interp = df_interp.round({'MD':1, 'TVD_SCS':1, 'TST':1})

    print('Start joining')
    def well_bal_interp_join(dataset):
        df_tst = df[df.TST.notna()].round({'MD':1})
        data_fu = df_tst[['well','MD','FORMATION_up', 'FORMATION', 'Field']]
        well_join = dataset.set_index(['well','MD']).join(data_fu.set_index(['well','MD'])).reset_index()
        well_join.insert(3, 'FORMATION_up', well_join.pop('FORMATION_up'))
        well_join.insert(4, 'FORMATION', well_join.pop('FORMATION'))
        well_join.insert(5, 'tst_index', well_join.pop('tst_index'))
        return well_join
    well_interp_v2 = well_bal_interp_join(df_interp)
    return well_interp_v2
df_interp = interpolate_by_depth_fm_run(df, 0.1)

In [ ]:
df[df.well == 'A01Y'][['well','MD','TST']]

In [ ]:
df_interp[df_interp.well == 'A01Y']

In [240]:
def phit_cleaning(df):
    md_total = df.groupby(['well','FORMATION_up'])['MD'].count().reset_index()
    phit_total = df.groupby(['well','FORMATION_up'])['PHIT'].count().reset_index()
    vsh_total = df.groupby(['well','FORMATION_up'])['VSH'].count().reset_index()
    full_df = md_total.set_index(['well','FORMATION_up']).join(phit_total.set_index(['well','FORMATION_up'])).reset_index()
    full_df_v2 = full_df.set_index(['well','FORMATION_up']).join(vsh_total.set_index(['well','FORMATION_up'])).reset_index()
    full_df_v2['phit%'] = full_df_v2['PHIT'] / full_df_v2['MD']
    full_df_v2['vsh%'] = full_df_v2['VSH'] / full_df_v2['MD']
    full_df_v2 = full_df_v2.round({'phit%':2, 'vsh%':2})
    phit_clean_bal8 = full_df_v2[(full_df_v2['phit%'] >=0.9) & (full_df_v2.FORMATION_up == 'Balakhany VIII')].well.unique()
    phit_clean_bal10 = full_df_v2[(full_df_v2['phit%'] >=0.9) & (full_df_v2.FORMATION_up == 'Balakhany X')].well.unique()
    return phit_clean_bal8, phit_clean_bal10
phit_clean_bal8, phit_clean_bal10 = phit_cleaning(df_interp)

def vsh_cleaning(df):
    md_total = df.groupby(['well','FORMATION_up'])['MD'].count().reset_index()
    phit_total = df.groupby(['well','FORMATION_up'])['PHIT'].count().reset_index()
    vsh_total = df.groupby(['well','FORMATION_up'])['VSH'].count().reset_index()
    full_df = md_total.set_index(['well','FORMATION_up']).join(phit_total.set_index(['well','FORMATION_up'])).reset_index()
    full_df_v2 = full_df.set_index(['well','FORMATION_up']).join(vsh_total.set_index(['well','FORMATION_up'])).reset_index()
    full_df_v2['phit%'] = full_df_v2['PHIT'] / full_df_v2['MD']
    full_df_v2['vsh%'] = full_df_v2['VSH'] / full_df_v2['MD']
    full_df_v2 = full_df_v2.round({'phit%':2, 'vsh%':2})
    vsh_clean_bal8 = full_df_v2[(full_df_v2['vsh%'] >=0.9) & (full_df_v2.FORMATION_up == 'Balakhany VIII')].well.unique()
    vsh_clean_bal10 = full_df_v2[(full_df_v2['vsh%'] >=0.9) & (full_df_v2.FORMATION_up == 'Balakhany X')].well.unique()
    return vsh_clean_bal8, vsh_clean_bal10
vsh_clean_bal8, vsh_clean_bal10 = vsh_cleaning(df_interp)

In [ ]:
phit_clean_bal8

## Diagramns

In [255]:
df_interp.loc[df_interp.NET_VSH !=0, 'NET_VSH'] = 1

In [ ]:
dataset = df_interp
wellname = 'A01Y'
fmname = 'Balakhany VIII'

# dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
dataset[(dataset.well==wellname)]

In [ ]:
def well_display_khtst(dataset, wellname, fmname, net_var, comments, ref_depth, fm_flag, depth_step, kh_include, print):
    """
    dataset = df_bal or something else
    net_var = NET or FLUIDS_int
    comments = put what you want
    ref_depth = MD or TST
    fm_flag = 1 if you need a FORMATION_up, 0 if just a simple FORMATION
    depth_step = step for ticks on the diagramm
    kh_include = 1 if we have KHtst in dataset, 0 if there is not KHtst
    print = 1 if we want to print the plot
    """
    if fm_flag == 0:
        data = dataset[(dataset.well==wellname) & (dataset.FORMATION == fmname)]
    if fm_flag == 1:
        data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT'] 
    net = data[net_var]
    perm = data['LPERM']
    if kh_include == 1:
        kh = data['KHtst']
    else:
        data['KHtst'] = 0
        kh = data['KHtst']
    fig, ax = plt.subplots(1,4, figsize=(7,7), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth+10, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    ax[0].grid(axis='y')
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.5, zorder=5)
    twin0.set_xlim(0, 1.5)
    ax[1].plot(rhob, depth, color='red') 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].set_xlim(1.65, 2.65)
    ax[1].grid(axis='y'), ax[1].grid(axis='x')
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 7, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue')
    twin1.set_xlim(0.6, 0)
    # ax[2].plot(rdeep, depth, color='black'), ax[2].set_xscale('log'), ax[2].set_xlim(0.1, 50), ax[2].invert_yaxis(), ax[2].grid(axis='x', which='both')
    ax[2].plot(phit, depth, color='green', linestyle='dashed'), ax[2].set_xlim(0.3, 0), ax[2].grid(axis='x') 
    ax[2].invert_yaxis()
    ax[2].grid(axis='y')
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed')
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5)
    twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    ax[3].plot(perm, depth, color='purple', alpha=0.66), ax[3].set_xscale('log'), ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    ax[3].grid(axis='y')
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    twin4 = ax[3].twiny()
    twin4.plot(kh, depth, color='black', alpha=1)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.tight_layout()
    if print == 1:
        path = 'C:\\jupyter\\SPP\\inputoutput\\wellplots\\'
        fig.savefig(path + fmname.replace(' ','') + '_' + wellname + '.png')
    else:
        pass

well_display_khtst(df, 'C13Z', 'Balakhany VIII', 'NET_VSH', 'test', 'MD', 1, 10, 0, 0)

## Experiments to find wrong data

In [139]:
def data_upload_v2(path, filename):
    df = pd.read_csv(path + filename)
    df = df.rename(columns={'wellName':'well', 'FORMATION_JOIN_BEST':'FORMATION', 'FORMATION_TOPS_AZPU':'TOPS_AZPU'})
    df['X_traj'] = df['SurfaceX'] + df['XOFFSET']
    df['Y_traj'] = df['SurfaceY'] + df['YOFFSET']
    return df
df_v2 = data_upload_v2('C:\\jupyter\\SPP\\input\\', 'newTechlogData_allWells.csv')

In [ ]:
test = df_v2.groupby(['well'])[['MD','GR_N','TST']].count().reset_index()
test['rel_gr'] = test['GR_N'] / test['MD']
test['rel_tst'] = test['TST'] / test['MD']
test[test.rel_tst <= 0.9 ].well.unique()

In [175]:
check_well_list_md = test[test.MD <= 100 ]

In [177]:
df_v2[df_v2.well.isin(check_well_list_md.well.unique())].to_csv('check_well_list_md.csv', index=False)

In [ ]:
df_v2[df_v2.well.isin(check_well_list_md.well.unique())].head(15)

In [ ]:
df_v2[df_v2.well.str.startswith('J')].well.unique()